# PRIMMDebug Log Data Analysis Notebook
This notebook displays all of the analysis of the log data that took place in the PRIMMDebug initial research paper.

The log data was collected from five schools between December 2024-February 2025. It is divided into the following sections:
1. **Summary statistics:** ...
2. **Establishing variables:**...
3. **Visualisation of variables:**...
4. **Students' written responses:**...

All you need to do is run the notebooks in order and the statistics that appear in the paper will be displayed. If there are any issues, please report them in the [Issues section of the GitHub repository](https://github.com/LaurieGale10/primmdebug-log-data-analysis/issues).

Before we run anything else, let's first import all of the necessary files.

In [1]:
from classes.ExerciseLog import ExerciseLog
from classes.StageLog import StageLog
from classes.StudentId import StudentId
from classes.exercise_classes.Exercise import Exercise
from classes.processors.ExerciseLogProcessor import ExerciseLogProcessor
from classes.processors.StageLogProcessor import StageLogProcessor

from loading_services.fetch_log_from_firebase import *
from loading_services.fetch_logs_from_file import fetch_data_from_json

from constants import *

from loading_services.parse_logs import *
import plotly.express as px
import datetime

exercises: list[Exercise] = parse_exercises(fetch_data_from_json("data/exercises"))
stage_logs: list[StageLog] = parse_stage_logs(fetch_data_from_json("data/stage_logs"))
exercise_logs: list[ExerciseLog] = parse_exercise_logs(stage_logs, fetch_data_from_json("data/exercise_logs"))
student_ids: list[StudentId] = parse_student_ids(fetch_data_from_json("data/student_ids"))

## Summary Statistics

### Exercise/Stage Logs
This data displays the following summary statistics to give information into the scale of the data we collected. We report below on:
- Number of exercises (that contain at least one completed PRIMMDebug stage)
  - Successful
  - Unsuccessful
  - Completed
  - Per each PRIMMDebug challenge
- Number of PRIMMDebug stages.
- Time of data collection


In [ ]:
print(f"Number of attempted PRIMMDebug challenges: {len(exercise_logs)}")

number_successful_exercises: int = 0
print(f"- Number of PRIMMDebug challenges where students reported successfully resolving the error they contained: {number_successful_exercises}")

number_unsuccessful_exercises: int = 0
print(f"- Number of PRIMMDebug challenges where students did not report successfully resolving the error they contained: {number_unsuccessful_exercises}")

number_completed_exercises: int = len([exercise_log for exercise_log in exercise_logs if ExerciseLogProcessor.get_last_stage(exercise_log).stage_name == DebuggingStage.modify])
number_completed_exercises_percentage: float = (number_completed_exercises / len(exercise_logs)) * 100
print(f"- Number of entirely completed PRIMMDebug challenges (where students reached the Make stage of PRIMMDebug): {number_completed_exercises}/{len(exercise_logs)} ({number_completed_exercises_percentage:.2f}%)\n")

challenge_attempts: dict[str] = {}
for exercise_log in exercise_logs:
    if exercise_log.exercise_name not in challenge_attempts:
        challenge_attempts[exercise_log.exercise_name] = 1
    else:
        challenge_attempts[exercise_log.exercise_name] += 1
challenge_attempts_fig = px.bar(x = challenge_attempts.keys(), y = challenge_attempts.values(), labels = {"x": "Challenge Name", "y": "Frequency"})
challenge_attempts_fig.show()

from collections import Counter
challenge_end_stages: dict[str, int] = dict(Counter([ExerciseLogProcessor.get_last_stage(exercise_log).stage_name.name for exercise_log in exercise_logs if ExerciseLogProcessor.get_last_stage(exercise_log) is not None]))
final_stage_fig = px.bar(x = list(challenge_end_stages.keys()), y = list(challenge_end_stages.values()), labels = {"x": "Final stage of PRIMMDebug", "y": "Frequency"})
final_stage_fig.show()

print(f"Number of completed PRIMMDebug stages: {len(stage_logs)}")

exercises_per_student: dict[str, int] = {}
for exercise in exercise_logs:
    student_id: str = exercise.student_id
    exercises_per_student[student_id] = exercises_per_student.get(student_id) + 1 if student_id in exercises_per_student else 1

attempted_challenges_per_student_fig = px.histogram(exercises_per_student.values(), marginal="box")
attempted_challenges_per_student_fig.show()

### Student Demographics

Number of students:
- By gender
- By year group
- By school


In [ ]:
print(f"Number of participating students: {len(student_ids)}")

gender_split_fig = px.bar(x = get_gender_split().keys(), y = get_gender_split().values(), labels = {"x": "Gender", "y": "Frequency"})
gender_split_fig.show()

year_group_split_fig = px.bar(x = get_year_group_split().keys(), y = get_year_group_split().values(), labels={"x": "Year Group", "y": "Frequency"})
year_group_split_fig.show()

school_split_fig = px.bar(x = get_school_split().keys(), y = get_school_split().values(), labels={"x": "School", "y": "Frequency"})
school_split_fig.show()

## Establishing Variables
Now we move onto introducing the variables that underpin our log data analysis. These include:

### Time Taken
- Per challenge attempt
- Per stage

In [ ]:
print("Time taken (seconds)")
time_per_challenge_fig = px.histogram([ExerciseLogProcessor.get_time_on_exercise(exercise) for exercise in exercise_logs if hasattr(exercise,"end_time")], marginal="box", labels={"x": "Time taken (seconds)"})
time_per_challenge_fig.show()
time_per_stage_fig = px.histogram([StageLogProcessor.get_time_on_stage(stage) for stage in stage_logs if StageLogProcessor.get_time_on_stage(stage) is not None], marginal="box", labels={"x": "Time taken (seconds)"})
time_per_stage_fig.show()

### Correctness of exercise
- Per challenge
- Per student

In [ ]:
print(" Correctness of PRIMMDebug challenges:")
print(f"- Per PRIMMDebug challenge")
print(f"- Per student")

### Number of stages taken for a PRIMMDebug challenge
- Per exercise
- Per student

In [ ]:
print(" Number of stages taken on a PRIMMDebug challenge:")
stages_per_challenge_fig = px.histogram([len(exercise.stage_logs) for exercise in exercise_logs], marginal="box", labels={"x": "Number of stages"})
stages_per_challenge_fig.show()
#TODO: Segregate by each specific challenge (and student?)

## Exercise Log Stats
Placeholder for exercise log stats

In [ ]:
total_time: float = sum([ExerciseLogProcessor.get_time_on_exercise(exercise_log) for exercise_log in exercise_logs])
print(f"Total time on PRIMMDebug exercises: {datetime.timedelta(seconds=total_time)}")

final_program_states: list[bool] = [ExerciseLogProcessor.is_final_program_erroneous(exercise) for exercise in exercise_logs]
successful_final_program_states: list[bool] = [final_program_state for final_program_state in final_program_states if final_program_state]
proportion_successful_final_program_states: float = (len(successful_final_program_states) / len(final_program_states)) * 100
print(f"Proportion of PRIMMDebug challenges where last program run successfully executed: {proportion_successful_final_program_states:.2f}%")

print("Time spent focused on PRIMMDebug window per exercise")
time_spent_focused: list[float] = [ExerciseLogProcessor.get_time_focused(exercise) for exercise in exercise_logs]
time_spent_focused_fig = px.histogram(time_spent_focused, marginal="box", labels={"x": "100% Time spent focused on PRIMMDebug window"})
time_spent_focused_fig.show()

exercises_with_test_case_views: int = len([exercise_log for exercise_log in exercise_logs if ExerciseLogProcessor.were_test_cases_viewed(exercise_log)]) / len(exercise_logs) * 100
print(f"Percentage of exercises where test cases were viewed at some point: {exercises_with_test_case_views:.2f}%")
print(f"- In the Inspect the Code stage: {len([exercise_log for exercise_log in exercise_logs if ExerciseLogProcessor.were_test_cases_viewed(exercise_log, [DebuggingStage.inspect_code])]) / len(exercise_logs) * 100}")
print(f"- In the Test stage: {len([exercise_log for exercise_log in exercise_logs if ExerciseLogProcessor.were_test_cases_viewed(exercise_log, [DebuggingStage.test])]) / len(exercise_logs) * 100}")


## Stage Log Stats
Placeholder for stage log stats

In [ ]:
number_inspect_code_stages: int = len([stage_log for stage_log in stage_logs if stage_log.stage_name == DebuggingStage.inspect_code])
number_no_response_inspect_code_stages: int = len([stage_log for stage_log in stage_logs if stage_log.stage_name == DebuggingStage.inspect_code and StageLogProcessor.does_inspect_the_code_contain_response(stage_log) is False])
print(f"Number of inspect the code stages which contain no response: {number_no_response_inspect_code_stages}/{number_inspect_code_stages} ({(number_no_response_inspect_code_stages / number_inspect_code_stages) * 100:.2f}%)")


find_error_stages_with_correct_field: list[StageLog] = [stage_log for stage_log in stage_logs if stage_log.stage_name == DebuggingStage.find_error and stage_log.correct is not None]
correct_find_error_stages: int = len([stage_log for stage_log in find_error_stages_with_correct_field if stage_log.correct])
print(f"Number of find the error stages where the correct response was entered (for challenges where students had to pinpoint a line): {correct_find_error_stages}/{len(find_error_stages_with_correct_field)} ({(correct_find_error_stages / len(find_error_stages_with_correct_field)) * 100:.2f}%)")


## Written Responses

For now, just group written responses by stage name and investigate them. Also get some stats on written responses for context

In [ ]:
from save_logs import *

import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize

nltk.download("words", quiet=True)
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)

#save_written_responses(exercise_logs)

english_words = set(words.words("en"))  # Load English words into a set for fast lookup
written_responses: list[str] = [response for exercise_responses in [ExerciseLogProcessor.get_written_responses(exercise_log) for exercise_log in exercise_logs] for response in exercise_responses]
print(f"Number of written responses: {len(written_responses)}")

responses_with_valid_words: list[str] = []
responses_with_invalid_words: list[str] = []

for response in written_responses:
    tokens = word_tokenize(response.lower())  # Convert to lowercase for case-insensitive matching
    # Check if any token is a valid English word
    if any(token in english_words for token in tokens):
        responses_with_valid_words.append(response)
    else:
        responses_with_invalid_words.append(response)

print(f"Number of written responses that contain at least one valid English word: {len(responses_with_valid_words)}/{len(written_responses)} ({(len(responses_with_valid_words) / len(written_responses)) * 100:.2f}%)")

{'uncanopied', 'unrulableness', 'tidely', 'unafflictedly', 'exhaustibility', 'Amizilis', 'frolic', 'theocentric', 'gaupus', 'styloauricularis', 'unweakened', 'milligal', 'sigillarist', 'spatialize', 'intraoctave', 'mesomerism', 'pseudolobar', 'quinquefoliolate', 'khilat', 'evaporate', 'farleu', 'decaesarize', 'hieroglyphist', 'conjecturally', 'anitrogenous', 'peregrinate', 'Renardine', 'monotypous', 'arboreous', 'notecase', 'rakshasa', 'reticulatogranulate', 'Boidae', 'armorwise', 'Jamesina', 'odeum', 'Dodonaean', 'emanative', 'superspecification', 'earlship', 'forestick', 'coverslut', 'privily', 'pedatiform', 'Eristalis', 'patriotics', 'pharmacic', 'chamar', 'unconformity', 'provisorily', 'carabidoid', 'cornberry', 'unintended', 'expiscate', 'undiffident', 'daze', 'unallow', 'rubidium', 'microcrystallogeny', 'geomancer', 'rapiered', 'dispart', 'Ilysiidae', 'spout', 'philomathical', 'planlessly', 'educationist', 'chilalgia', 'prehaustorium', 'coroneted', 'dewily', 'cotyledonal', 'Myrme

Stage-specific stuff:
- Success rate of students who didn't write anything for inspect the code